Author: Ronny F. Pacheco Date: Oct 2024
Copyright: © 2024 Ronny Pacheco License: MIT License

---

MIT License

Copyright (c) 2024 Ronny Pacheco

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.    

# Needed modules

In [1]:
# Load the needed libraries
import pickle
import os
from tokenize import group  # type: ignore

import pandas as pd

In [2]:
# https://kioku-space.com/en/jupyter-skip-execution/
from IPython.core.magic import register_cell_magic # type: ignore


@register_cell_magic
def skip(line, cell):
    print('Skipping cell')
    if line and cell:
        pass
    return

# Pickle save

In [3]:
%%skip
# =============================================================================
# main function
# =============================================================================
def data_save_load(option):
    """
    This function is used to save or load data for the jupyter notebook
    """
    path_folder = "ipynb_db"  # Folder to save variables
    os.makedirs(path_folder, exist_ok=True)  # Create folder if not exist
    # notebook_name = os.path.basename(os.path.abspath(''))
    notebook_name = '1.3.leishmania_repeated_elements_filtering'
    path_file = os.path.join(path_folder, f"{notebook_name}.variables.pkl") # Path to save the variables

    if option == "save":
        with open(path_file, "wb") as pickle_file:
            dict_variables = {
                'good_lre' : good_lre,
                'reduced_lre' : reduced_lre,
                'lre_df' : lre_df,
            }
            pickle.dump(dict_variables, pickle_file)
    elif option == "load":
        with open(path_file, "rb") as pickle_file:
            variables = pickle.load(pickle_file)
        # Now load the variables
        for pickle_key, pickle_value in variables.items():
            print(f"* Loading variable: {pickle_key}")
            globals()[pickle_key] = pickle_value
# =============================================================================
# Call the function
# =============================================================================
data_save_load(option="load")

Skipping cell


# Prepare data

In [4]:
lre_df = pd.read_csv('./data_1.1/lre_to_filter.csv', sep=',', header=0)
print(lre_df.shape)
print(lre_df.dtypes)
lre_df.head()

(881, 16)
neg_name         object
chrom            object
start             int64
end               int64
strand           object
gene_id          object
transcript_id    object
parent_id        object
gene               bool
transcript         bool
CDS                bool
3utr               bool
5utr               bool
pseudogen        object
notes            object
interval         object
dtype: object


,neg_name,chrom,start,end,strand,gene_id,transcript_id,parent_id,gene,transcript,CDS,3utr,5utr,pseudogen,notes,interval
0,lre_2.1,LinJ.02,93941,102762,-,LINF_020007000,NaN,NaN,True,False,False,False,False,NaN,phosphoglycan_beta_1-3_galactosyltransferase,"[93941, 102762]"
1,lre_2.1,LinJ.02,93941,102762,-,NaN,LINF_02T0007000,LINF_020007000,False,True,False,False,False,NaN,phosphoglycan_beta_1-3_galactosyltransferase,"[93941, 102762]"
2,lre_2.1,LinJ.02,93941,99642,-,NaN,NaN,LINF_02T0007000,False,False,False,True,False,NaN,NaN,"[93941, 99642]"
3,lre_2.2,LinJ.02,137296,139716,-,LINF_020007950,NaN,NaN,True,False,False,False,False,NaN,NaN,"[137296, 139716]"
4,lre_2.2,LinJ.02,137296,139716,-,NaN,LINF_02T0007950,LINF_020007950,False,True,False,False,False,NaN,NaN,"[137296, 139716]"


In [5]:
# Change name from 'neg_name' to 'lre_name'
lre_df.rename(columns={'neg_name': 'lre_name'}, inplace=True)

In [6]:
# Let's add a new column about the "neg_family"
# let's take the number "x" from lre_x.1, where 'x' is the number to add to a new column

# Define a function to extract the number x from neg_name column
def extract_neg_group(column_name):
    import re
    match = re.search(r'lre_(\d+)\.\d+', column_name)
    return int(match.group(1)) if match else None

# Apply the function to create a new column
lre_df['lre_family'] = lre_df['lre_name'].apply(extract_neg_group)

# Display the updated dataframe
lre_df.head()

,lre_name,chrom,start,end,strand,gene_id,transcript_id,parent_id,gene,transcript,CDS,3utr,5utr,pseudogen,notes,interval,lre_family
0,lre_2.1,LinJ.02,93941,102762,-,LINF_020007000,NaN,NaN,True,False,False,False,False,NaN,phosphoglycan_beta_1-3_galactosyltransferase,"[93941, 102762]",2
1,lre_2.1,LinJ.02,93941,102762,-,NaN,LINF_02T0007000,LINF_020007000,False,True,False,False,False,NaN,phosphoglycan_beta_1-3_galactosyltransferase,"[93941, 102762]",2
2,lre_2.1,LinJ.02,93941,99642,-,NaN,NaN,LINF_02T0007000,False,False,False,True,False,NaN,NaN,"[93941, 99642]",2
3,lre_2.2,LinJ.02,137296,139716,-,LINF_020007950,NaN,NaN,True,False,False,False,False,NaN,NaN,"[137296, 139716]",2
4,lre_2.2,LinJ.02,137296,139716,-,NaN,LINF_02T0007950,LINF_020007950,False,True,False,False,False,NaN,NaN,"[137296, 139716]",2


# Load functions

In [7]:
def search_string(data_frame, searching_string):
    """
    :param data_frame: The DataFrame containing the data to be searched.
    :param searching_string: The string to search for within the 'notes' column of the DataFrame.
    :return: A filtered DataFrame containing rows where 'notes' contain the search string (case-insensitive).
    """
    filtered_df = data_frame[data_frame['notes'].fillna('').str.contains(searching_string, case=False)]
    print(f"The of filtered data: {filtered_df.shape}")
    print(f"The unique values in column 'lre_name': {filtered_df['lre_name'].nunique()}")
    return filtered_df

def filter_and_display(data_frame, search_term):
    """
    :param data_frame: The DataFrame containing the data to be filtered.
    :param search_term: The term to search for within the DataFrame.
    :return: A filtered DataFrame with specific columns where the "gene" column is True.
    """
    filtered_data = search_string(data_frame, search_term)
    print("=" * 50)
    print(filtered_data['notes'].value_counts())
    print("=" * 50)
    filtered_result = filtered_data.loc[
        filtered_data["gene"] == True,
        ['lre_name', 'lre_family', 'chrom', 'gene_id', 'notes', 'start', 'end']
    ]
    return filtered_result

def checking_data(data_frame):
    """
    :param data_frame: A pandas DataFrame that contains the dataset to be checked.
    :return: None. The function prints the shape of the DataFrame and the number of unique values in the 'lre_name' column.
    """
    print(f"Shape of the data frame is: {data_frame.shape}")
    print(f"Number of unique values in column 'lre_name': {data_frame['lre_name'].nunique()} ")

def check_family(data_frame, family_name):
    """
    :param data_frame: The input pandas DataFrame containing genomic data with columns such as 'lre_name', 'chrom', 'gene_id', 'notes', 'start', and 'end'.
    :param family_name: The family name pattern (string) to be searched at the end of 'lre_name'.
    :return: A DataFrame filtered to include rows where 'lre_name' contains the given family name pattern at the end and 'gene' is True. The returned DataFrame includes columns 'lre_name', 'chrom', 'gene_id', 'notes', 'start', and 'end'.
    """
    pattern = fr'\d+{family_name}$'
    filtered_data = data_frame.loc[
        (data_frame['lre_name'].str.contains(pattern, na=False)) & (data_frame['gene'] == True),
        ['lre_name', 'chrom', 'gene_id', 'notes', 'start', 'end']
    ]
    return filtered_data
 

def group_and_count(data_frame, group_column='lre_name'):
    """
    :param data_frame: The pandas DataFrame containing the data to be processed.
    :param group_column: The column name on which to group the DataFrame.
    :return: A DataFrame containing the count of unique 'notes' values, sorted in descending order.
    """
    grouped_df = data_frame.groupby(group_column)
    grouped_column_counter = {}
    loc_counter_global = {}
    for _, group_data in grouped_df:
        notes = group_data['notes'].unique()
        for element in notes:
            if element is not None:
                if element not in grouped_column_counter:
                    grouped_column_counter[element] = 1
                else:
                    grouped_column_counter[element] += 1
                    
        loc_data = group_data[["gene", "transcript", "CDS", "3utr", "5utr"]].sum()
        for loc_element in loc_data.index:
            if loc_data[loc_element] > 0:
                if loc_element not in loc_counter_global:
                    loc_counter_global[loc_element] = 1
                else:
                    loc_counter_global[loc_element] += 1
        
    notes_counter_global_sorted = dict(sorted(grouped_column_counter.items(), key=lambda x: x[1], reverse=True))
    notes_counter_global_sorted_df = pd.DataFrame(notes_counter_global_sorted.items(), columns=["notes", "count"])
    print(loc_counter_global)
    return notes_counter_global_sorted_df

def accept_group(data_frame, filtered_data, good_df, keep_filtering_df):
    """
    :param data_frame: The main DataFrame that contains data to be processed.
    :param filtered_data: A DataFrame containing the subset of `data_frame` that meets certain criteria.
    :param good_df: A DataFrame that accumulates 'good' data from `data_frame`.
    :param keep_filtering_df: A DataFrame that requires further filtering.
    :return: A tuple containing `good_df` and `keep_filtering_df`. 
    """
    print('='*50)
    print('Data to filter:')
    all_data = data_frame[data_frame['lre_name'].isin(filtered_data['lre_name'])]
    checking_data(all_data)
    
    if good_df is not None:
        if not good_df.empty:
            print('='*50)
            print('Good lre elements:')
            good_df = pd.concat([good_df, all_data])
            checking_data(good_df)
    else:  # If it doesn't exist
        good_df = all_data
        print('='*50)
        print('Good lre elements:')
        checking_data(good_df)
    
    print('='*50)
    print('Keep filtering df:')
    keep_filtering_df = keep_filtering_df[~keep_filtering_df['lre_name'].isin(filtered_data['lre_name'])]
    checking_data(keep_filtering_df)
        
    return good_df, keep_filtering_df

# Filtering

## Descriptive stats

In [8]:
checking_data(lre_df)
group_and_count(lre_df, 'lre_name')

Shape of the data frame is: (881, 17)
Number of unique values in column 'lre_name': 307 
{'gene': 307, 'transcript': 307, '3utr': 184, '5utr': 20, 'CDS': 2}


,notes,count
0,NaN,307
1,hypothetical_protein_-_conserved,32
2,amastin_surface_glycoprotein_-_putative,23
3,protein_of_unknown_function_-_conserved,8
4,glucose_transporter,8
...,...,...
89,tRNA-Leu,1
90,engulfment_and_cell_motility_domain_2_-_putative,1
91,Protein_kinase_domain-containing_protein,1
92,protein-l-isoaspartate_o-methyltransferase_-_p...,1


## Filtering

### hypothetical_protein

In [9]:
filter_df = filter_and_display(lre_df, 'hypothetical_protein')
filter_df

The of filtered data: (74, 17)
The unique values in column 'lre_name': 37
notes
hypothetical_protein_-_conserved    64
hypothetical_protein                10
Name: count, dtype: int64


,lre_name,lre_family,chrom,gene_id,notes,start,end
10,lre_3.3,3,LinJ.04,LINF_040010300,hypothetical_protein_-_conserved,208282,210473
25,lre_5.2,5,LinJ.05,LINF_050009400,hypothetical_protein_-_conserved,144533,148377
124,lre_19.2,19,LinJ.12,LINF_120012900,hypothetical_protein_-_conserved,393788,397393
130,lre_19.3,19,LinJ.12,LINF_120013300,hypothetical_protein_-_conserved,406344,409449
184,lre_24.2,24,LinJ.15,LINF_150014000,hypothetical_protein_-_conserved,306215,309145
202,lre_27.2,27,LinJ.17,LINF_170019100,hypothetical_protein_-_conserved,580021,585507
205,lre_26.2,26,LinJ.17,LINF_170022200,hypothetical_protein_-_conserved,687597,689967
210,lre_28.1,28,LinJ.19,LINF_190009700,hypothetical_protein_-_conserved,182279,187638
237,lre_32.1,32,LinJ.19,LINF_190019900,hypothetical_protein_-_conserved,615371,617193
260,lre_34.1,34,LinJ.23,LINF_230013900,hypothetical_protein,286043,296768


In [10]:
filter_df['lre_family'].value_counts()

lre_family
19    2
44    2
49    2
61    2
96    2
26    1
27    1
3     1
32    1
28    1
36    1
34    1
41    1
46    1
5     1
24    1
48    1
47    1
60    1
50    1
64    1
65    1
66    1
58    1
69    1
81    1
87    1
86    1
89    1
92    1
95    1
98    1
Name: count, dtype: int64

In [11]:
good_lre, reduced_lre = accept_group(lre_df, filter_df, None, lre_df)

Data to filter:
Shape of the data frame is: (121, 17)
Number of unique values in column 'lre_name': 37 
Good lre elements:
Shape of the data frame is: (121, 17)
Number of unique values in column 'lre_name': 37 
Keep filtering df:
Shape of the data frame is: (760, 17)
Number of unique values in column 'lre_name': 270 


In [12]:
group_and_count(reduced_lre)

{'gene': 270, 'transcript': 270, '3utr': 151, '5utr': 14, 'CDS': 2}


,notes,count
0,NaN,270
1,amastin_surface_glycoprotein_-_putative,23
2,protein_of_unknown_function_-_conserved,8
3,glucose_transporter,8
4,amastin-like_protein,7
...,...,...
85,tRNA-Leu,1
86,engulfment_and_cell_motility_domain_2_-_putative,1
87,Protein_kinase_domain-containing_protein,1
88,protein-l-isoaspartate_o-methyltransferase_-_p...,1


### amastin_surface_glycoprotein

In [13]:
filter_df = filter_and_display(reduced_lre, 'amastin_surface_glycoprotein')
filter_df

The of filtered data: (50, 17)
The unique values in column 'lre_name': 25
notes
amastin_surface_glycoprotein_-_putative    46
amastin_surface_glycoprotein                4
Name: count, dtype: int64


,lre_name,lre_family,chrom,gene_id,notes,start,end
605,lre_73.9,73,LinJ.34,LINF_340024200,amastin_surface_glycoprotein_-_putative,805687,808315
641,lre_77.1,77,LinJ.34,LINF_340034300,amastin_surface_glycoprotein_-_putative,1240773,1245780
643,lre_78.1,78,LinJ.34,LINF_340034300,amastin_surface_glycoprotein_-_putative,1240773,1245780
645,lre_79.1,79,LinJ.34,LINF_340034300,amastin_surface_glycoprotein_-_putative,1240773,1245780
650,lre_77.2,77,LinJ.34,LINF_340034400,amastin_surface_glycoprotein_-_putative,1246328,1249925
652,lre_78.2,78,LinJ.34,LINF_340034400,amastin_surface_glycoprotein_-_putative,1246328,1249925
654,lre_79.2,79,LinJ.34,LINF_340034400,amastin_surface_glycoprotein_-_putative,1246328,1249925
656,lre_80.1,80,LinJ.34,LINF_340034400,amastin_surface_glycoprotein_-_putative,1246328,1249925
664,lre_77.3,77,LinJ.34,LINF_340034500,amastin_surface_glycoprotein_-_putative,1252201,1255794
666,lre_78.3,78,LinJ.34,LINF_340034500,amastin_surface_glycoprotein_-_putative,1252201,1255794


Let's check all the families involved:

In [14]:
filter_df['lre_family'].value_counts()

lre_family
78    7
79    7
77    6
80    4
73    1
Name: count, dtype: int64

There are 5 families involved: 78, 79, 77, 80 and 73 

Checking family 78 with 7 members:

In [15]:
lre_df[(lre_df['gene']==True) & (lre_df['lre_family'] == 78)]

,lre_name,chrom,start,end,strand,gene_id,transcript_id,parent_id,gene,transcript,CDS,3utr,5utr,pseudogen,notes,interval,lre_family
643,lre_78.1,LinJ.34,1240773,1245780,+,LINF_340034300,NaN,NaN,True,False,False,False,False,NaN,amastin_surface_glycoprotein_-_putative,"[1240773, 1245780]",78
652,lre_78.2,LinJ.34,1246328,1249925,+,LINF_340034400,NaN,NaN,True,False,False,False,False,NaN,amastin_surface_glycoprotein_-_putative,"[1246328, 1249925]",78
666,lre_78.3,LinJ.34,1252201,1255794,+,LINF_340034500,NaN,NaN,True,False,False,False,False,NaN,amastin_surface_glycoprotein_-_putative,"[1252201, 1255794]",78
680,lre_78.4,LinJ.34,1258088,1261692,+,LINF_340034600,NaN,NaN,True,False,False,False,False,NaN,amastin_surface_glycoprotein_-_putative,"[1258088, 1261692]",78
694,lre_78.5,LinJ.34,1263977,1267600,+,LINF_340034700,NaN,NaN,True,False,False,False,False,NaN,amastin_surface_glycoprotein_-_putative,"[1263977, 1267600]",78
706,lre_78.6,LinJ.34,1269875,1273542,+,LINF_340034800,NaN,NaN,True,False,False,False,False,NaN,amastin_surface_glycoprotein,"[1269875, 1273542]",78
716,lre_78.7,LinJ.34,1275817,1279464,+,LINF_340034900,NaN,NaN,True,False,False,False,False,NaN,amastin_surface_glycoprotein_-_putative,"[1275817, 1279464]",78


All family the 7 family members of "78" are involved. So we can reject them.

In [16]:
# Remove family 77 from "reduced_lre"
reduced_lre = reduced_lre[reduced_lre['lre_family'] != 78]

Let's check family 79 with 7 members involved in 'amastin'

In [17]:
# Check family 79
lre_df[(lre_df['gene']==True) & (lre_df['lre_family'] == 79)]

,lre_name,chrom,start,end,strand,gene_id,transcript_id,parent_id,gene,transcript,CDS,3utr,5utr,pseudogen,notes,interval,lre_family
645,lre_79.1,LinJ.34,1240773,1245780,+,LINF_340034300,NaN,NaN,True,False,False,False,False,NaN,amastin_surface_glycoprotein_-_putative,"[1240773, 1245780]",79
654,lre_79.2,LinJ.34,1246328,1249925,+,LINF_340034400,NaN,NaN,True,False,False,False,False,NaN,amastin_surface_glycoprotein_-_putative,"[1246328, 1249925]",79
668,lre_79.3,LinJ.34,1252201,1255794,+,LINF_340034500,NaN,NaN,True,False,False,False,False,NaN,amastin_surface_glycoprotein_-_putative,"[1252201, 1255794]",79
682,lre_79.4,LinJ.34,1258088,1261692,+,LINF_340034600,NaN,NaN,True,False,False,False,False,NaN,amastin_surface_glycoprotein_-_putative,"[1258088, 1261692]",79
696,lre_79.5,LinJ.34,1263977,1267600,+,LINF_340034700,NaN,NaN,True,False,False,False,False,NaN,amastin_surface_glycoprotein_-_putative,"[1263977, 1267600]",79
708,lre_79.6,LinJ.34,1269875,1273542,+,LINF_340034800,NaN,NaN,True,False,False,False,False,NaN,amastin_surface_glycoprotein,"[1269875, 1273542]",79
718,lre_79.7,LinJ.34,1275817,1279464,+,LINF_340034900,NaN,NaN,True,False,False,False,False,NaN,amastin_surface_glycoprotein_-_putative,"[1275817, 1279464]",79


All of hte 7 members are involved. Reject them

In [18]:
# Remove this family from "reduced_lre"
reduced_lre = reduced_lre[reduced_lre['lre_family'] != 79]

Check the family 77 with 6 members involved:

In [19]:
# Check family 77
lre_df[(lre_df['gene']==True) & (lre_df['lre_family'] == 77)]

,lre_name,chrom,start,end,strand,gene_id,transcript_id,parent_id,gene,transcript,CDS,3utr,5utr,pseudogen,notes,interval,lre_family
641,lre_77.1,LinJ.34,1240773,1245780,+,LINF_340034300,NaN,NaN,True,False,False,False,False,NaN,amastin_surface_glycoprotein_-_putative,"[1240773, 1245780]",77
650,lre_77.2,LinJ.34,1246328,1249925,+,LINF_340034400,NaN,NaN,True,False,False,False,False,NaN,amastin_surface_glycoprotein_-_putative,"[1246328, 1249925]",77
664,lre_77.3,LinJ.34,1252201,1255794,+,LINF_340034500,NaN,NaN,True,False,False,False,False,NaN,amastin_surface_glycoprotein_-_putative,"[1252201, 1255794]",77
678,lre_77.4,LinJ.34,1258088,1261692,+,LINF_340034600,NaN,NaN,True,False,False,False,False,NaN,amastin_surface_glycoprotein_-_putative,"[1258088, 1261692]",77
692,lre_77.5,LinJ.34,1263977,1267600,+,LINF_340034700,NaN,NaN,True,False,False,False,False,NaN,amastin_surface_glycoprotein_-_putative,"[1263977, 1267600]",77
714,lre_77.6,LinJ.34,1275817,1279464,+,LINF_340034900,NaN,NaN,True,False,False,False,False,NaN,amastin_surface_glycoprotein_-_putative,"[1275817, 1279464]",77


All the 6 members all involved. Reject them

In [20]:
# Remove family '77' from 'reduced_lre'
reduced_lre = reduced_lre[reduced_lre['lre_family'] != 77]

Check family 80 with 4 members involved:

In [21]:
# Check family '80'
lre_df[(lre_df['gene']==True) & (lre_df['lre_family'] == 80)]

,lre_name,chrom,start,end,strand,gene_id,transcript_id,parent_id,gene,transcript,CDS,3utr,5utr,pseudogen,notes,interval,lre_family
656,lre_80.1,LinJ.34,1246328,1249925,+,LINF_340034400,NaN,NaN,True,False,False,False,False,NaN,amastin_surface_glycoprotein_-_putative,"[1246328, 1249925]",80
670,lre_80.2,LinJ.34,1252201,1255794,+,LINF_340034500,NaN,NaN,True,False,False,False,False,NaN,amastin_surface_glycoprotein_-_putative,"[1252201, 1255794]",80
684,lre_80.3,LinJ.34,1258088,1261692,+,LINF_340034600,NaN,NaN,True,False,False,False,False,NaN,amastin_surface_glycoprotein_-_putative,"[1258088, 1261692]",80
698,lre_80.4,LinJ.34,1263977,1267600,+,LINF_340034700,NaN,NaN,True,False,False,False,False,NaN,amastin_surface_glycoprotein_-_putative,"[1263977, 1267600]",80


All 4 members are involved. Reject them.

In [22]:
# Remove family '80' from 'reduced_lre'
reduced_lre = reduced_lre[reduced_lre['lre_family'] != 80]

Check family 73 involved with 1 member

In [23]:
# Check family 73
lre_df.loc[
    (lre_df['gene']==True) & (lre_df['lre_family'] == 73),
    ['lre_name', 'chrom', 'gene_id', 'notes', 'start', 'end']
]

,lre_name,chrom,gene_id,notes,start,end
559,lre_73.1,LinJ.34,LINF_340023220,NaN,754280,757294
565,lre_73.1,LinJ.34,LINF_340023260,NaN,758277,761292
567,lre_73.2,LinJ.34,LINF_340023260,NaN,758277,761292
573,lre_73.2,LinJ.34,LINF_340023280,NaN,761293,765197
575,lre_73.3,LinJ.34,LINF_340023280,NaN,761293,765197
577,lre_73.3,LinJ.34,LINF_340023300,amastin-like_surface_protein_-_putative,766090,768423
582,lre_73.4,LinJ.34,LINF_340023415,NaN,772123,775066
584,lre_73.4,LinJ.34,LINF_340023430,NaN,775900,777200
590,lre_73.5,LinJ.34,LINF_340023445,NaN,777201,778297
592,lre_73.7,LinJ.34,LINF_340023460,NaN,778924,780014


There are 7 members but only one with a function. The others may be 'amastin' as well?

Let's save only this family 73

In [24]:
# Take this family 73 for 'good_lre'
filter_df = reduced_lre[(reduced_lre['gene']==True) & (reduced_lre['lre_family'] == 73)]

In [25]:
filter_df['lre_name'].unique()

array(['lre_73.1', 'lre_73.2', 'lre_73.3', 'lre_73.4', 'lre_73.5',
       'lre_73.7', 'lre_73.8', 'lre_73.9'], dtype=object)

The member 73.6 is not here, because it's in the intergenic data.

In [26]:
good_lre, reduced_lre = accept_group(reduced_lre, filter_df, good_lre, reduced_lre)

Data to filter:
Shape of the data frame is: (31, 17)
Number of unique values in column 'lre_name': 8 
Good lre elements:
Shape of the data frame is: (152, 17)
Number of unique values in column 'lre_name': 45 
Keep filtering df:
Shape of the data frame is: (657, 17)
Number of unique values in column 'lre_name': 238 


In [27]:
group_and_count(reduced_lre)

{'gene': 238, 'transcript': 238, '3utr': 124, '5utr': 14, 'CDS': 2}


,notes,count
0,NaN,238
1,protein_of_unknown_function_-_conserved,8
2,glucose_transporter,8
3,amastin-like_protein,7
4,Tripartite_attachment_complex_40|TAC40,6
...,...,...
82,tRNA-Leu,1
83,engulfment_and_cell_motility_domain_2_-_putative,1
84,Protein_kinase_domain-containing_protein,1
85,protein-l-isoaspartate_o-methyltransferase_-_p...,1


### protein_of_unknown_function

In [28]:
filter_df = filter_and_display(reduced_lre, 'protein_of_unknown_function')
filter_df

The of filtered data: (24, 17)
The unique values in column 'lre_name': 11
notes
protein_of_unknown_function_-_conserved            16
protein_of_unknown_function_(DUF3184)               4
Protein_of_unknown_function_(DUF962),_putative      2
Protein_of_unknown_function_(DUF778)_-_putative     2
Name: count, dtype: int64


,lre_name,lre_family,chrom,gene_id,notes,start,end
115,lre_19.1,19,LinJ.12,LINF_120012350,"Protein_of_unknown_function_(DUF962),_putative",372226,374050
118,lre_19.1,19,LinJ.12,LINF_120012400,protein_of_unknown_function_-_conserved,374051,377775
141,lre_19.5,19,LinJ.12,LINF_120013800,protein_of_unknown_function_-_conserved,429222,431627
149,lre_20.1,20,LinJ.12,LINF_120016200,protein_of_unknown_function_-_conserved,528652,535332
151,lre_21.1,21,LinJ.12,LINF_120016200,protein_of_unknown_function_-_conserved,528652,535332
189,lre_25.1,25,LinJ.16,LINF_160015800,protein_of_unknown_function_(DUF3184),371773,376703
192,lre_25.2,25,LinJ.16,LINF_160015820,protein_of_unknown_function_(DUF3184),377060,383329
219,lre_29.2,29,LinJ.19,LINF_190010600,protein_of_unknown_function_-_conserved,224264,231735
323,lre_44.2,44,LinJ.27,LINF_270015300,Protein_of_unknown_function_(DUF778)_-_putative,396554,398111
456,lre_63.1,63,LinJ.31,LINF_310026900,protein_of_unknown_function_-_conserved,973364,979320


In [29]:
filter_df['lre_family'].value_counts()

lre_family
19    3
25    2
20    1
21    1
29    1
44    1
63    1
72    1
91    1
Name: count, dtype: int64

Since we don't know their functions. Let's accept them.

In [30]:
good_lre, reduced_lre = accept_group(reduced_lre, filter_df, good_lre, reduced_lre)

Data to filter:
Shape of the data frame is: (39, 17)
Number of unique values in column 'lre_name': 11 
Good lre elements:
Shape of the data frame is: (191, 17)
Number of unique values in column 'lre_name': 56 
Keep filtering df:
Shape of the data frame is: (618, 17)
Number of unique values in column 'lre_name': 227 


In [31]:
group_and_count(reduced_lre)

{'gene': 227, 'transcript': 227, '3utr': 113, '5utr': 12, 'CDS': 2}


,notes,count
0,NaN,227
1,glucose_transporter,8
2,amastin-like_protein,7
3,Tripartite_attachment_complex_40|TAC40,6
4,Ketoacyl-CoA_synthase|Fatty_acid_elongase|ELO-3,5
...,...,...
77,tRNA-Leu,1
78,engulfment_and_cell_motility_domain_2_-_putative,1
79,Protein_kinase_domain-containing_protein,1
80,protein-l-isoaspartate_o-methyltransferase_-_p...,1


### glucose_transporter

In [32]:
filter_df = filter_and_display(reduced_lre, 'glucose_transporter')
filter_df

The of filtered data: (16, 17)
The unique values in column 'lre_name': 8
notes
glucose_transporter    16
Name: count, dtype: int64


,lre_name,lre_family,chrom,gene_id,notes,start,end
840,lre_97.1,97,LinJ.36,LINF_360072900,glucose_transporter,2456799,2460093
843,lre_97.2,97,LinJ.36,LINF_360073000,glucose_transporter,2460371,2463651
846,lre_97.3,97,LinJ.36,LINF_360073100,glucose_transporter,2463918,2467210
849,lre_97.4,97,LinJ.36,LINF_360073200,glucose_transporter,2467477,2470767
852,lre_97.5,97,LinJ.36,LINF_360073300,glucose_transporter,2471045,2474323
855,lre_97.6,97,LinJ.36,LINF_360073400,glucose_transporter,2474601,2477880
858,lre_97.7,97,LinJ.36,LINF_360073500,glucose_transporter,2478158,2481440
861,lre_97.8,97,LinJ.36,LINF_360073600,glucose_transporter,2481878,2486596


Let's check all the elements in the family 97

In [33]:
lre_df[(lre_df['gene']==True) & (lre_df['lre_family'] == 97)]

,lre_name,chrom,start,end,strand,gene_id,transcript_id,parent_id,gene,transcript,CDS,3utr,5utr,pseudogen,notes,interval,lre_family
840,lre_97.1,LinJ.36,2456799,2460093,-,LINF_360072900,NaN,NaN,True,False,False,False,False,NaN,glucose_transporter,"[2456799, 2460093]",97
843,lre_97.2,LinJ.36,2460371,2463651,-,LINF_360073000,NaN,NaN,True,False,False,False,False,NaN,glucose_transporter,"[2460371, 2463651]",97
846,lre_97.3,LinJ.36,2463918,2467210,-,LINF_360073100,NaN,NaN,True,False,False,False,False,NaN,glucose_transporter,"[2463918, 2467210]",97
849,lre_97.4,LinJ.36,2467477,2470767,-,LINF_360073200,NaN,NaN,True,False,False,False,False,NaN,glucose_transporter,"[2467477, 2470767]",97
852,lre_97.5,LinJ.36,2471045,2474323,-,LINF_360073300,NaN,NaN,True,False,False,False,False,NaN,glucose_transporter,"[2471045, 2474323]",97
855,lre_97.6,LinJ.36,2474601,2477880,-,LINF_360073400,NaN,NaN,True,False,False,False,False,NaN,glucose_transporter,"[2474601, 2477880]",97
858,lre_97.7,LinJ.36,2478158,2481440,-,LINF_360073500,NaN,NaN,True,False,False,False,False,NaN,glucose_transporter,"[2478158, 2481440]",97
861,lre_97.8,LinJ.36,2481878,2486596,-,LINF_360073600,NaN,NaN,True,False,False,False,False,NaN,glucose_transporter,"[2481878, 2486596]",97


It caught the repeated elements in 3'UTR of "glucose_transporter", so let's reject them.

In [34]:
# Remove from 'reduced_lre' all elements from the family 97
reduced_lre = reduced_lre[reduced_lre['lre_family'] != 97]
checking_data(reduced_lre)

Shape of the data frame is: (594, 17)
Number of unique values in column 'lre_name': 219 


In [35]:
group_and_count(reduced_lre)

{'gene': 219, 'transcript': 219, '3utr': 105, '5utr': 12, 'CDS': 2}


,notes,count
0,NaN,219
1,amastin-like_protein,7
2,Tripartite_attachment_complex_40|TAC40,6
3,Ketoacyl-CoA_synthase|Fatty_acid_elongase|ELO-3,5
4,amastin-like_surface_protein_-_putative,5
...,...,...
76,tRNA-Leu,1
77,engulfment_and_cell_motility_domain_2_-_putative,1
78,Protein_kinase_domain-containing_protein,1
79,protein-l-isoaspartate_o-methyltransferase_-_p...,1


### amastin-like_protein

In [36]:
filter_df = filter_and_display(reduced_lre, 'amastin-like')
filter_df

The of filtered data: (24, 17)
The unique values in column 'lre_name': 12
notes
amastin-like_protein                       14
amastin-like_surface_protein_-_putative    10
Name: count, dtype: int64


,lre_name,lre_family,chrom,gene_id,notes,start,end
81,lre_14.2,14,LinJ.08,LINF_080012900,amastin-like_protein,337654,340312
525,lre_70.1,70,LinJ.34,LINF_340015400,amastin-like_surface_protein_-_putative,423724,426392
530,lre_70.3,70,LinJ.34,LINF_340015500,amastin-like_protein,432504,435210
533,lre_70.4,70,LinJ.34,LINF_340015800,amastin-like_protein,445685,448392
536,lre_70.5,70,LinJ.34,LINF_340015900,amastin-like_protein,450104,453599
541,lre_71.2,71,LinJ.34,LINF_340017500,amastin-like_protein,513172,516089
549,lre_70.6,70,LinJ.34,LINF_340022900,amastin-like_protein,739819,743551
552,lre_71.3,71,LinJ.34,LINF_340022900,amastin-like_protein,739819,743551
608,lre_72.12,72,LinJ.34,LINF_340024500,amastin-like_surface_protein_-_putative,814218,816590
611,lre_72.11,72,LinJ.34,LINF_340024500,amastin-like_surface_protein_-_putative,814218,816590


In [37]:
filter_df['lre_family'].value_counts()

lre_family
70    5
72    4
71    2
14    1
Name: count, dtype: int64

Let's check the family 70

In [38]:
lre_df.loc[
    (lre_df['gene']==True) & (lre_df['lre_family'] == 70),
    ['lre_name', 'chrom', 'gene_id', 'notes', 'start', 'end']
]

,lre_name,chrom,gene_id,notes,start,end
525,lre_70.1,LinJ.34,LINF_340015400,amastin-like_surface_protein_-_putative,423724,426392
528,lre_70.2,LinJ.34,LINF_340015450,NaN,428116,431093
530,lre_70.3,LinJ.34,LINF_340015500,amastin-like_protein,432504,435210
533,lre_70.4,LinJ.34,LINF_340015800,amastin-like_protein,445685,448392
536,lre_70.5,LinJ.34,LINF_340015900,amastin-like_protein,450104,453599
549,lre_70.6,LinJ.34,LINF_340022900,amastin-like_protein,739819,743551


We can't rejec them because one is 'NaN' for some reason.

Let's check family 72

In [39]:
lre_df.loc[
    (lre_df['gene']==True) & (lre_df['lre_family'] == 72),
    ['lre_name', 'chrom', 'gene_id', 'notes', 'start', 'end']
]

,lre_name,chrom,gene_id,notes,start,end
544,lre_72.1,LinJ.34,LINF_340018600,protein_of_unknown_function_-_conserved,557626,562306
555,lre_72.2,LinJ.34,LINF_340023220,NaN,754280,757294
557,lre_72.3,LinJ.34,LINF_340023220,NaN,754280,757294
561,lre_72.4,LinJ.34,LINF_340023260,NaN,758277,761292
563,lre_72.5,LinJ.34,LINF_340023260,NaN,758277,761292
569,lre_72.6,LinJ.34,LINF_340023280,NaN,761293,765197
571,lre_72.7,LinJ.34,LINF_340023280,NaN,761293,765197
580,lre_72.8,LinJ.34,LINF_340023415,NaN,772123,775066
586,lre_72.9,LinJ.34,LINF_340023430,NaN,775900,777200
588,lre_72.9,LinJ.34,LINF_340023445,NaN,777201,778297


Can't reject them. Could be a lot of information for those NaN values

Let's check family 71

In [40]:
lre_df.loc[
    (lre_df['gene']==True) & (lre_df['lre_family'] == 71),
    ['lre_name', 'chrom', 'gene_id', 'notes', 'start', 'end']
]

,lre_name,chrom,gene_id,notes,start,end
523,lre_71.1,LinJ.34,LINF_340015350,NaN,422379,423723
539,lre_71.2,LinJ.34,LINF_340017475,NaN,511354,513171
541,lre_71.2,LinJ.34,LINF_340017500,amastin-like_protein,513172,516089
547,lre_71.3,LinJ.34,LINF_340022850,NaN,735203,739818
552,lre_71.3,LinJ.34,LINF_340022900,amastin-like_protein,739819,743551


Can't reject them. Could be a lot of information for those NaN values

In [41]:
lre_df.loc[
    (lre_df['gene']==True) & (lre_df['lre_family'] == 73),
    ['lre_name', 'chrom', 'gene_id', 'notes', 'start', 'end']
]

,lre_name,chrom,gene_id,notes,start,end
559,lre_73.1,LinJ.34,LINF_340023220,NaN,754280,757294
565,lre_73.1,LinJ.34,LINF_340023260,NaN,758277,761292
567,lre_73.2,LinJ.34,LINF_340023260,NaN,758277,761292
573,lre_73.2,LinJ.34,LINF_340023280,NaN,761293,765197
575,lre_73.3,LinJ.34,LINF_340023280,NaN,761293,765197
577,lre_73.3,LinJ.34,LINF_340023300,amastin-like_surface_protein_-_putative,766090,768423
582,lre_73.4,LinJ.34,LINF_340023415,NaN,772123,775066
584,lre_73.4,LinJ.34,LINF_340023430,NaN,775900,777200
590,lre_73.5,LinJ.34,LINF_340023445,NaN,777201,778297
592,lre_73.7,LinJ.34,LINF_340023460,NaN,778924,780014


Can't reject them. Could be a lot of information for those NaN values

In [42]:
lre_df.loc[
    (lre_df['gene']==True) & (lre_df['lre_family'] == 14),
    ['lre_name', 'chrom', 'gene_id', 'notes', 'start', 'end']
]

,lre_name,chrom,gene_id,notes,start,end
73,lre_14.1,LinJ.08,LINF_080012850,NaN,332481,335118
81,lre_14.2,LinJ.08,LINF_080012900,amastin-like_protein,337654,340312


Let's save everything in `filter_df'

In [43]:
good_lre, reduced_lre = accept_group(reduced_lre, filter_df, good_lre, reduced_lre)

Data to filter:
Shape of the data frame is: (40, 17)
Number of unique values in column 'lre_name': 12 
Good lre elements:
Shape of the data frame is: (231, 17)
Number of unique values in column 'lre_name': 68 
Keep filtering df:
Shape of the data frame is: (554, 17)
Number of unique values in column 'lre_name': 207 


In [44]:
group_and_count(reduced_lre)

{'gene': 207, 'transcript': 207, '3utr': 94, '5utr': 11, 'CDS': 2}


,notes,count
0,NaN,207
1,Tripartite_attachment_complex_40|TAC40,6
2,Ketoacyl-CoA_synthase|Fatty_acid_elongase|ELO-3,5
3,argininosuccinate_synthase|Citrulline-aspartat...,3
4,beta_galactofuranosyl_transferase|LPG1,3
...,...,...
74,tRNA-Leu,1
75,engulfment_and_cell_motility_domain_2_-_putative,1
76,Protein_kinase_domain-containing_protein,1
77,protein-l-isoaspartate_o-methyltransferase_-_p...,1


### Tripartite_attachment_complex_40|TAC40

In [45]:
filter_df = filter_and_display(reduced_lre, 'Tripartite_attachment_complex_40|TAC40')
filter_df

The of filtered data: (12, 17)
The unique values in column 'lre_name': 6
notes
Tripartite_attachment_complex_40|TAC40    12
Name: count, dtype: int64


,lre_name,lre_family,chrom,gene_id,notes,start,end
731,lre_82.1,82,LinJ.34,LINF_340037600,Tripartite_attachment_complex_40|TAC40,1372364,1376363
733,lre_83.1,83,LinJ.34,LINF_340037600,Tripartite_attachment_complex_40|TAC40,1372364,1376363
735,lre_81.3,81,LinJ.34,LINF_340037600,Tripartite_attachment_complex_40|TAC40,1372364,1376363
742,lre_82.2,82,LinJ.34,LINF_340038100,Tripartite_attachment_complex_40|TAC40,1391972,1395962
744,lre_83.2,83,LinJ.34,LINF_340038100,Tripartite_attachment_complex_40|TAC40,1391972,1395962
746,lre_81.5,81,LinJ.34,LINF_340038100,Tripartite_attachment_complex_40|TAC40,1391972,1395962


In [46]:
filter_df['lre_family'].value_counts()

lre_family
82    2
83    2
81    2
Name: count, dtype: int64

In [47]:
# Checking family 82
lre_df.loc[
    (lre_df['gene']==True) & (lre_df['lre_family'] == 82),
    ['lre_name', 'chrom', 'gene_id', 'notes', 'start', 'end']
]

,lre_name,chrom,gene_id,notes,start,end
731,lre_82.1,LinJ.34,LINF_340037600,Tripartite_attachment_complex_40|TAC40,1372364,1376363
742,lre_82.2,LinJ.34,LINF_340038100,Tripartite_attachment_complex_40|TAC40,1391972,1395962


In [48]:
# Checking family 83
lre_df.loc[
    (lre_df['gene']==True) & (lre_df['lre_family'] == 83),
    ['lre_name', 'chrom', 'gene_id', 'notes', 'start', 'end']
]

,lre_name,chrom,gene_id,notes,start,end
733,lre_83.1,LinJ.34,LINF_340037600,Tripartite_attachment_complex_40|TAC40,1372364,1376363
744,lre_83.2,LinJ.34,LINF_340038100,Tripartite_attachment_complex_40|TAC40,1391972,1395962


In [49]:
# Check family 81
lre_df.loc[
    (lre_df['gene']==True) & (lre_df['lre_family'] == 81),
    ['lre_name', 'chrom', 'gene_id', 'notes', 'start', 'end']
]

,lre_name,chrom,gene_id,notes,start,end
725,lre_81.1,LinJ.34,LINF_340037300,mitochondrial_carrier_protein|MCP6,1354107,1357619
728,lre_81.2,LinJ.34,LINF_340037350,hypothetical_protein_-_conserved,1358089,1365799
735,lre_81.3,LinJ.34,LINF_340037600,Tripartite_attachment_complex_40|TAC40,1372364,1376363
740,lre_81.4,LinJ.34,LINF_340037850,NaN,1381967,1383514
746,lre_81.5,LinJ.34,LINF_340038100,Tripartite_attachment_complex_40|TAC40,1391972,1395962
751,lre_81.7,LinJ.34,LINF_340038800,NAD_dependent_epimerase/dehydratase_family,1420070,1428451


In [50]:
# Remove family 82
reduced_lre = reduced_lre[reduced_lre['lre_family'] != 82]

# Remove family 83
reduced_lre = reduced_lre[reduced_lre['lre_family'] != 83]

In [51]:
# Let's only keep family 81
filter_df = reduced_lre[reduced_lre['lre_family'] == 81]

In [52]:
# Let's keep this data
good_lre, reduced_lre = accept_group(reduced_lre, filter_df, good_lre, reduced_lre)

Data to filter:
Shape of the data frame is: (14, 17)
Number of unique values in column 'lre_name': 5 
Good lre elements:
Shape of the data frame is: (245, 17)
Number of unique values in column 'lre_name': 73 
Keep filtering df:
Shape of the data frame is: (528, 17)
Number of unique values in column 'lre_name': 198 


In [53]:
group_and_count(reduced_lre)

{'gene': 198, 'transcript': 198, '3utr': 86, '5utr': 11, 'CDS': 2}


,notes,count
0,NaN,198
1,Ketoacyl-CoA_synthase|Fatty_acid_elongase|ELO-3,5
2,argininosuccinate_synthase|Citrulline-aspartat...,3
3,beta_galactofuranosyl_transferase|LPG1,3
4,alpha/beta_hydrolase,2
...,...,...
71,tRNA-Leu,1
72,engulfment_and_cell_motility_domain_2_-_putative,1
73,Protein_kinase_domain-containing_protein,1
74,protein-l-isoaspartate_o-methyltransferase_-_p...,1


### Ketoacyl-CoA_synthase|Fatty_acid_elongase|ELO-3

In [54]:
filter_df = filter_and_display(reduced_lre, 'Ketoacyl-CoA_synthase|Fatty_acid_elongase|ELO-3')
filter_df

The of filtered data: (10, 17)
The unique values in column 'lre_name': 5
notes
Ketoacyl-CoA_synthase|Fatty_acid_elongase|ELO-3    10
Name: count, dtype: int64


,lre_name,lre_family,chrom,gene_id,notes,start,end
163,lre_22.1,22,LinJ.14,LINF_140012400,Ketoacyl-CoA_synthase|Fatty_acid_elongase|ELO-3,255152,267139
166,lre_22.2,22,LinJ.14,LINF_140012400,Ketoacyl-CoA_synthase|Fatty_acid_elongase|ELO-3,255152,267139
169,lre_22.3,22,LinJ.14,LINF_140012400,Ketoacyl-CoA_synthase|Fatty_acid_elongase|ELO-3,255152,267139
172,lre_22.4,22,LinJ.14,LINF_140012400,Ketoacyl-CoA_synthase|Fatty_acid_elongase|ELO-3,255152,267139
175,lre_22.5,22,LinJ.14,LINF_140012400,Ketoacyl-CoA_synthase|Fatty_acid_elongase|ELO-3,255152,267139


In [55]:
filter_df['lre_family'].value_counts()

lre_family
22    5
Name: count, dtype: int64

In [56]:
lre_df.loc[
    (lre_df['gene']==True) & (lre_df['lre_family'] == 22),
    ['lre_name', 'chrom', 'gene_id', 'notes', 'start', 'end']
]

,lre_name,chrom,gene_id,notes,start,end
163,lre_22.1,LinJ.14,LINF_140012400,Ketoacyl-CoA_synthase|Fatty_acid_elongase|ELO-3,255152,267139
166,lre_22.2,LinJ.14,LINF_140012400,Ketoacyl-CoA_synthase|Fatty_acid_elongase|ELO-3,255152,267139
169,lre_22.3,LinJ.14,LINF_140012400,Ketoacyl-CoA_synthase|Fatty_acid_elongase|ELO-3,255152,267139
172,lre_22.4,LinJ.14,LINF_140012400,Ketoacyl-CoA_synthase|Fatty_acid_elongase|ELO-3,255152,267139
175,lre_22.5,LinJ.14,LINF_140012400,Ketoacyl-CoA_synthase|Fatty_acid_elongase|ELO-3,255152,267139
178,lre_22.6,LinJ.14,LINF_140012850,NaN,285611,287461


In [57]:
# Let's keep them
good_lre, reduced_lre = accept_group(reduced_lre, filter_df, good_lre, reduced_lre)

Data to filter:
Shape of the data frame is: (15, 17)
Number of unique values in column 'lre_name': 5 
Good lre elements:
Shape of the data frame is: (260, 17)
Number of unique values in column 'lre_name': 78 
Keep filtering df:
Shape of the data frame is: (513, 17)
Number of unique values in column 'lre_name': 193 


In [58]:
group_and_count(reduced_lre)

{'gene': 193, 'transcript': 193, '3utr': 81, '5utr': 11, 'CDS': 2}


,notes,count
0,NaN,193
1,argininosuccinate_synthase|Citrulline-aspartat...,3
2,beta_galactofuranosyl_transferase|LPG1,3
3,alpha/beta_hydrolase,2
4,Methionine_aminopeptidase_1|MetAP1,2
...,...,...
70,tRNA-Leu,1
71,engulfment_and_cell_motility_domain_2_-_putative,1
72,Protein_kinase_domain-containing_protein,1
73,protein-l-isoaspartate_o-methyltransferase_-_p...,1


### argininosuccinate_synthase|Citrulline-aspartate_ligase

In [59]:
filter_df = filter_and_display(reduced_lre, 'argininosuccinate_synthase|Citrulline-aspartate_ligase')
filter_df

The of filtered data: (6, 17)
The unique values in column 'lre_name': 3
notes
argininosuccinate_synthase|Citrulline-aspartate_ligase    6
Name: count, dtype: int64


,lre_name,lre_family,chrom,gene_id,notes,start,end
251,lre_33.1,33,LinJ.23,LINF_230007900,argininosuccinate_synthase|Citrulline-aspartat...,93890,100632
253,lre_33.2,33,LinJ.23,LINF_230007900,argininosuccinate_synthase|Citrulline-aspartat...,93890,100632
255,lre_33.3,33,LinJ.23,LINF_230007900,argininosuccinate_synthase|Citrulline-aspartat...,93890,100632


In [60]:
lre_df.loc[
    (lre_df['gene']==True) & (lre_df['lre_family'] == 33),
    ['lre_name', 'chrom', 'gene_id', 'notes', 'start', 'end']
]

,lre_name,chrom,gene_id,notes,start,end
251,lre_33.1,LinJ.23,LINF_230007900,argininosuccinate_synthase|Citrulline-aspartat...,93890,100632
253,lre_33.2,LinJ.23,LINF_230007900,argininosuccinate_synthase|Citrulline-aspartat...,93890,100632
255,lre_33.3,LinJ.23,LINF_230007900,argininosuccinate_synthase|Citrulline-aspartat...,93890,100632


In [61]:
# Let's keep them
good_lre, reduced_lre = accept_group(reduced_lre, filter_df, good_lre, reduced_lre)

Data to filter:
Shape of the data frame is: (9, 17)
Number of unique values in column 'lre_name': 3 
Good lre elements:
Shape of the data frame is: (269, 17)
Number of unique values in column 'lre_name': 81 
Keep filtering df:
Shape of the data frame is: (504, 17)
Number of unique values in column 'lre_name': 190 


In [62]:
group_and_count(reduced_lre)

{'gene': 190, 'transcript': 190, '3utr': 78, '5utr': 11, 'CDS': 2}


,notes,count
0,NaN,190
1,beta_galactofuranosyl_transferase|LPG1,3
2,alpha/beta_hydrolase,2
3,Methionine_aminopeptidase_1|MetAP1,2
4,ATG8/AUT7/APG8/PAZ2,2
...,...,...
69,tRNA-Leu,1
70,engulfment_and_cell_motility_domain_2_-_putative,1
71,Protein_kinase_domain-containing_protein,1
72,protein-l-isoaspartate_o-methyltransferase_-_p...,1


### beta_galactofuranosyl_transferase|LPG1

In [63]:
filter_df = filter_and_display(reduced_lre, 'beta_galactofuranosyl_transferase|LPG1')
filter_df

The of filtered data: (6, 17)
The unique values in column 'lre_name': 3
notes
beta_galactofuranosyl_transferase|LPG1    6
Name: count, dtype: int64


,lre_name,lre_family,chrom,gene_id,notes,start,end
280,lre_38.1,38,LinJ.25,LINF_250005000,beta_galactofuranosyl_transferase|LPG1,799,5363
283,lre_39.1,39,LinJ.25,LINF_250005000,beta_galactofuranosyl_transferase|LPG1,799,5363
286,lre_37.1,37,LinJ.25,LINF_250005000,beta_galactofuranosyl_transferase|LPG1,799,5363


In [64]:
filter_df['lre_family'].value_counts()

lre_family
38    1
39    1
37    1
Name: count, dtype: int64

In [65]:
# Check family 38
lre_df.loc[
    (lre_df['gene']==True) & (lre_df['lre_family'] == 38),
    ['lre_name', 'chrom', 'gene_id', 'notes', 'start', 'end']
]

,lre_name,chrom,gene_id,notes,start,end
280,lre_38.1,LinJ.25,LINF_250005000,beta_galactofuranosyl_transferase|LPG1,799,5363
289,lre_38.2,LinJ.25,LINF_250006300,electron_transfer_flavoprotein,29864,33171
295,lre_38.3,LinJ.25,LINF_250007950,NaN,81625,83195


In [66]:
# Check family 39
lre_df.loc[
    (lre_df['gene']==True) & (lre_df['lre_family'] == 39),
    ['lre_name', 'chrom', 'gene_id', 'notes', 'start', 'end']
]

,lre_name,chrom,gene_id,notes,start,end
283,lre_39.1,LinJ.25,LINF_250005000,beta_galactofuranosyl_transferase|LPG1,799,5363
297,lre_39.2,LinJ.25,LINF_250007950,NaN,81625,83195


In [67]:
# Check family 37
lre_df.loc[
    (lre_df['gene']==True) & (lre_df['lre_family'] == 37),
    ['lre_name', 'chrom', 'gene_id', 'notes', 'start', 'end']
]

,lre_name,chrom,gene_id,notes,start,end
286,lre_37.1,LinJ.25,LINF_250005000,beta_galactofuranosyl_transferase|LPG1,799,5363
521,lre_37.3,LinJ.33,LINF_330042920,NaN,1531735,1532106


In [68]:
# Let's keep them
good_lre, reduced_lre = accept_group(reduced_lre, filter_df, good_lre, reduced_lre)

Data to filter:
Shape of the data frame is: (9, 17)
Number of unique values in column 'lre_name': 3 
Good lre elements:
Shape of the data frame is: (278, 17)
Number of unique values in column 'lre_name': 84 
Keep filtering df:
Shape of the data frame is: (495, 17)
Number of unique values in column 'lre_name': 187 


In [69]:
group_and_count(reduced_lre)

{'gene': 187, 'transcript': 187, '3utr': 75, '5utr': 11, 'CDS': 2}


,notes,count
0,NaN,187
1,alpha/beta_hydrolase,2
2,Methionine_aminopeptidase_1|MetAP1,2
3,ATG8/AUT7/APG8/PAZ2,2
4,UDP-galactose_transporter|LPG5A,2
...,...,...
68,tRNA-Leu,1
69,engulfment_and_cell_motility_domain_2_-_putative,1
70,Protein_kinase_domain-containing_protein,1
71,protein-l-isoaspartate_o-methyltransferase_-_p...,1


### alpha/beta_hydrolase

In [70]:
filter_df = filter_and_display(reduced_lre, 'alpha/beta_hydrolase')
filter_df

The of filtered data: (4, 17)
The unique values in column 'lre_name': 2
notes
alpha/beta_hydrolase    4
Name: count, dtype: int64


,lre_name,lre_family,chrom,gene_id,notes,start,end
157,lre_20.2,20,LinJ.12,LINF_120017600,alpha/beta_hydrolase,581574,586501
159,lre_21.3,21,LinJ.12,LINF_120017600,alpha/beta_hydrolase,581574,586501


In [71]:
# Check family 20
lre_df.loc[
    (lre_df['gene']==True) & (lre_df['lre_family'] == 20),
    ['lre_name', 'chrom', 'gene_id', 'notes', 'start', 'end']
]

,lre_name,chrom,gene_id,notes,start,end
149,lre_20.1,LinJ.12,LINF_120016200,protein_of_unknown_function_-_conserved,528652,535332
157,lre_20.2,LinJ.12,LINF_120017600,alpha/beta_hydrolase,581574,586501


In [72]:
# Check family 21
lre_df.loc[
    (lre_df['gene']==True) & (lre_df['lre_family'] == 21),
    ['lre_name', 'chrom', 'gene_id', 'notes', 'start', 'end']
]

,lre_name,chrom,gene_id,notes,start,end
151,lre_21.1,LinJ.12,LINF_120016200,protein_of_unknown_function_-_conserved,528652,535332
155,lre_21.2,LinJ.12,LINF_120016450,NaN,545453,547275
159,lre_21.3,LinJ.12,LINF_120017600,alpha/beta_hydrolase,581574,586501


In [73]:
# Let's keep them
good_lre, reduced_lre = accept_group(reduced_lre, filter_df, good_lre, reduced_lre)

Data to filter:
Shape of the data frame is: (6, 17)
Number of unique values in column 'lre_name': 2 
Good lre elements:
Shape of the data frame is: (284, 17)
Number of unique values in column 'lre_name': 86 
Keep filtering df:
Shape of the data frame is: (489, 17)
Number of unique values in column 'lre_name': 185 


In [74]:
group_and_count(reduced_lre)

{'gene': 185, 'transcript': 185, '3utr': 73, '5utr': 11, 'CDS': 2}


,notes,count
0,NaN,185
1,Methionine_aminopeptidase_1|MetAP1,2
2,ATG8/AUT7/APG8/PAZ2,2
3,UDP-galactose_transporter|LPG5A,2
4,electron_transfer_flavoprotein,2
...,...,...
67,tRNA-Leu,1
68,engulfment_and_cell_motility_domain_2_-_putative,1
69,Protein_kinase_domain-containing_protein,1
70,protein-l-isoaspartate_o-methyltransferase_-_p...,1


### Methionine_aminopeptidase_1|MetAP1

In [75]:
filter_df = filter_and_display(reduced_lre, 'Methionine_aminopeptidase_1|MetAP1')
filter_df

The of filtered data: (4, 17)
The unique values in column 'lre_name': 2
notes
Methionine_aminopeptidase_1|MetAP1    4
Name: count, dtype: int64


,lre_name,lre_family,chrom,gene_id,notes,start,end
213,lre_29.1,29,LinJ.19,LINF_190010300,Methionine_aminopeptidase_1|MetAP1,207856,211386
215,lre_28.2,28,LinJ.19,LINF_190010300,Methionine_aminopeptidase_1|MetAP1,207856,211386


In [76]:
# Check family 29
lre_df.loc[
    (lre_df['gene']==True) & (lre_df['lre_family'] == 29),
    ['lre_name', 'chrom', 'gene_id', 'notes', 'start', 'end']
]

,lre_name,chrom,gene_id,notes,start,end
213,lre_29.1,LinJ.19,LINF_190010300,Methionine_aminopeptidase_1|MetAP1,207856,211386
219,lre_29.2,LinJ.19,LINF_190010600,protein_of_unknown_function_-_conserved,224264,231735


In [77]:
# Check family 28
lre_df.loc[
    (lre_df['gene']==True) & (lre_df['lre_family'] == 28),
    ['lre_name', 'chrom', 'gene_id', 'notes', 'start', 'end']
]

,lre_name,chrom,gene_id,notes,start,end
210,lre_28.1,LinJ.19,LINF_190009700,hypothetical_protein_-_conserved,182279,187638
215,lre_28.2,LinJ.19,LINF_190010300,Methionine_aminopeptidase_1|MetAP1,207856,211386


In [78]:
good_lre, reduced_lre = accept_group(reduced_lre, filter_df, good_lre, reduced_lre)

Data to filter:
Shape of the data frame is: (6, 17)
Number of unique values in column 'lre_name': 2 
Good lre elements:
Shape of the data frame is: (290, 17)
Number of unique values in column 'lre_name': 88 
Keep filtering df:
Shape of the data frame is: (483, 17)
Number of unique values in column 'lre_name': 183 


In [79]:
group_and_count(reduced_lre)

{'gene': 183, 'transcript': 183, '3utr': 71, '5utr': 11, 'CDS': 2}


,notes,count
0,NaN,183
1,ATG8/AUT7/APG8/PAZ2,2
2,UDP-galactose_transporter|LPG5A,2
3,electron_transfer_flavoprotein,2
4,sphingolipid_delta_4_desaturase,2
...,...,...
66,tRNA-Leu,1
67,engulfment_and_cell_motility_domain_2_-_putative,1
68,Protein_kinase_domain-containing_protein,1
69,protein-l-isoaspartate_o-methyltransferase_-_p...,1


### ATG8/AUT7/APG8/PAZ2

In [80]:
filter_df = filter_and_display(reduced_lre, 'ATG8/AUT7/APG8/PAZ2')
filter_df

The of filtered data: (4, 17)
The unique values in column 'lre_name': 2
notes
ATG8/AUT7/APG8/PAZ2    4
Name: count, dtype: int64


,lre_name,lre_family,chrom,gene_id,notes,start,end
222,lre_30.1,30,LinJ.19,LINF_190013900,ATG8/AUT7/APG8/PAZ2,361081,362554
225,lre_30.2,30,LinJ.19,LINF_190014100,ATG8/AUT7/APG8/PAZ2,364663,366138


In [81]:
# Check family 30
lre_df.loc[
    (lre_df['gene']==True) & (lre_df['lre_family'] == 30),
    ['lre_name', 'chrom', 'gene_id', 'notes', 'start', 'end']
]

,lre_name,chrom,gene_id,notes,start,end
222,lre_30.1,LinJ.19,LINF_190013900,ATG8/AUT7/APG8/PAZ2,361081,362554
225,lre_30.2,LinJ.19,LINF_190014100,ATG8/AUT7/APG8/PAZ2,364663,366138


In [82]:
# Let's keep them
good_lre, reduced_lre = accept_group(reduced_lre, filter_df, good_lre, reduced_lre)

Data to filter:
Shape of the data frame is: (6, 17)
Number of unique values in column 'lre_name': 2 
Good lre elements:
Shape of the data frame is: (296, 17)
Number of unique values in column 'lre_name': 90 
Keep filtering df:
Shape of the data frame is: (477, 17)
Number of unique values in column 'lre_name': 181 


In [83]:
group_and_count(reduced_lre)

{'gene': 181, 'transcript': 181, '3utr': 69, '5utr': 11, 'CDS': 2}


,notes,count
0,NaN,181
1,UDP-galactose_transporter|LPG5A,2
2,electron_transfer_flavoprotein,2
3,sphingolipid_delta_4_desaturase,2
4,Exonuclease_-_putative,2
...,...,...
65,tRNA-Leu,1
66,engulfment_and_cell_motility_domain_2_-_putative,1
67,Protein_kinase_domain-containing_protein,1
68,protein-l-isoaspartate_o-methyltransferase_-_p...,1


### UDP-galactose_transporter|LPG5A

In [84]:
filter_df = filter_and_display(reduced_lre, 'UDP-galactose_transporter|LPG5A')
filter_df

The of filtered data: (6, 17)
The unique values in column 'lre_name': 2
notes
UDP-galactose_transporter|LPG5A    6
Name: count, dtype: int64


,lre_name,lre_family,chrom,gene_id,notes,start,end
268,lre_36.1,36,LinJ.24,LINF_240008300,UDP-galactose_transporter|LPG5A,110205,117720
271,lre_36.2,36,LinJ.24,LINF_240008300,UDP-galactose_transporter|LPG5A,110205,117720
274,lre_36.2,36,LinJ.24,LINF_240008400,UDP-galactose_transporter|LPG5A,117721,122337


In [85]:
# Check family 36
lre_df.loc[
    (lre_df['gene']==True) & (lre_df['lre_family'] == 36),
    ['lre_name', 'chrom', 'gene_id', 'notes', 'start', 'end']
]

,lre_name,chrom,gene_id,notes,start,end
266,lre_36.1,LinJ.24,LINF_240008250,NaN,108132,110204
268,lre_36.1,LinJ.24,LINF_240008300,UDP-galactose_transporter|LPG5A,110205,117720
271,lre_36.2,LinJ.24,LINF_240008300,UDP-galactose_transporter|LPG5A,110205,117720
274,lre_36.2,LinJ.24,LINF_240008400,UDP-galactose_transporter|LPG5A,117721,122337
277,lre_36.3,LinJ.24,LINF_240009100,hypothetical_protein_-_conserved,145556,151128


In [86]:
# Let's save them
good_lre, reduced_lre = accept_group(reduced_lre, filter_df, good_lre, reduced_lre)

Data to filter:
Shape of the data frame is: (11, 17)
Number of unique values in column 'lre_name': 2 
Good lre elements:
Shape of the data frame is: (307, 17)
Number of unique values in column 'lre_name': 92 
Keep filtering df:
Shape of the data frame is: (466, 17)
Number of unique values in column 'lre_name': 179 


In [87]:
group_and_count(reduced_lre)

{'gene': 179, 'transcript': 179, '3utr': 68, '5utr': 9, 'CDS': 2}


,notes,count
0,NaN,179
1,electron_transfer_flavoprotein,2
2,sphingolipid_delta_4_desaturase,2
3,Exonuclease_-_putative,2
4,Rab-GTPase-TBC_domain_containing_protein,2
...,...,...
64,tRNA-Leu,1
65,engulfment_and_cell_motility_domain_2_-_putative,1
66,Protein_kinase_domain-containing_protein,1
67,protein-l-isoaspartate_o-methyltransferase_-_p...,1


### electron_transfer_flavoprotein

In [88]:
filter_df = filter_and_display(reduced_lre, 'electron_transfer_flavoprotein')
filter_df

The of filtered data: (4, 17)
The unique values in column 'lre_name': 2
notes
electron_transfer_flavoprotein    4
Name: count, dtype: int64


,lre_name,lre_family,chrom,gene_id,notes,start,end
289,lre_38.2,38,LinJ.25,LINF_250006300,electron_transfer_flavoprotein,29864,33171
292,lre_40.1,40,LinJ.25,LINF_250006300,electron_transfer_flavoprotein,29864,33171


In [89]:
# Check family 38
lre_df.loc[
    (lre_df['gene']==True) & (lre_df['lre_family'] == 38),
    ['lre_name', 'chrom', 'gene_id', 'notes', 'start', 'end']
]

,lre_name,chrom,gene_id,notes,start,end
280,lre_38.1,LinJ.25,LINF_250005000,beta_galactofuranosyl_transferase|LPG1,799,5363
289,lre_38.2,LinJ.25,LINF_250006300,electron_transfer_flavoprotein,29864,33171
295,lre_38.3,LinJ.25,LINF_250007950,NaN,81625,83195


In [90]:
# Check family 40
lre_df.loc[
    (lre_df['gene']==True) & (lre_df['lre_family'] == 40),
    ['lre_name', 'chrom', 'gene_id', 'notes', 'start', 'end']
]

,lre_name,chrom,gene_id,notes,start,end
292,lre_40.1,LinJ.25,LINF_250006300,electron_transfer_flavoprotein,29864,33171
299,lre_40.2,LinJ.25,LINF_250007950,NaN,81625,83195


In [91]:
# let's keep them
good_lre, reduced_lre = accept_group(reduced_lre, filter_df, good_lre, reduced_lre)

Data to filter:
Shape of the data frame is: (6, 17)
Number of unique values in column 'lre_name': 2 
Good lre elements:
Shape of the data frame is: (313, 17)
Number of unique values in column 'lre_name': 94 
Keep filtering df:
Shape of the data frame is: (460, 17)
Number of unique values in column 'lre_name': 177 


In [92]:
group_and_count(reduced_lre)

{'gene': 177, 'transcript': 177, '3utr': 66, '5utr': 9, 'CDS': 2}


,notes,count
0,NaN,177
1,sphingolipid_delta_4_desaturase,2
2,Exonuclease_-_putative,2
3,Rab-GTPase-TBC_domain_containing_protein,2
4,H(+)-exporting_diphosphatase,2
...,...,...
63,tRNA-Leu,1
64,engulfment_and_cell_motility_domain_2_-_putative,1
65,Protein_kinase_domain-containing_protein,1
66,protein-l-isoaspartate_o-methyltransferase_-_p...,1


### sphingolipid_delta_4_desaturase

In [93]:
filter_df = filter_and_display(reduced_lre, 'sphingolipid_delta_4_desaturase')
filter_df

The of filtered data: (4, 17)
The unique values in column 'lre_name': 2
notes
sphingolipid_delta_4_desaturase    4
Name: count, dtype: int64


,lre_name,lre_family,chrom,gene_id,notes,start,end
306,lre_42.2,42,LinJ.26,LINF_260022000,sphingolipid_delta_4_desaturase,605032,607263
309,lre_42.3,42,LinJ.26,LINF_260022100,sphingolipid_delta_4_desaturase,607528,609700


In [94]:
# Check family 42
lre_df.loc[
    (lre_df['gene']==True) & (lre_df['lre_family'] == 42),
    ['lre_name', 'chrom', 'gene_id', 'notes', 'start', 'end']
]

,lre_name,chrom,gene_id,notes,start,end
304,lre_42.1,LinJ.26,LINF_260021850,NaN,598501,599917
306,lre_42.2,LinJ.26,LINF_260022000,sphingolipid_delta_4_desaturase,605032,607263
309,lre_42.3,LinJ.26,LINF_260022100,sphingolipid_delta_4_desaturase,607528,609700


In [95]:
# Let's keep them
good_lre, reduced_lre = accept_group(reduced_lre, filter_df, good_lre, reduced_lre)

Data to filter:
Shape of the data frame is: (6, 17)
Number of unique values in column 'lre_name': 2 
Good lre elements:
Shape of the data frame is: (319, 17)
Number of unique values in column 'lre_name': 96 
Keep filtering df:
Shape of the data frame is: (454, 17)
Number of unique values in column 'lre_name': 175 


In [96]:
group_and_count(reduced_lre)

{'gene': 175, 'transcript': 175, '3utr': 66, '5utr': 7, 'CDS': 2}


,notes,count
0,NaN,175
1,Exonuclease_-_putative,2
2,Rab-GTPase-TBC_domain_containing_protein,2
3,H(+)-exporting_diphosphatase,2
4,peptidase_m20/m25/m40_family-like_protein,2
...,...,...
62,tRNA-Leu,1
63,engulfment_and_cell_motility_domain_2_-_putative,1
64,Protein_kinase_domain-containing_protein,1
65,protein-l-isoaspartate_o-methyltransferase_-_p...,1


### Exonuclease_-_putative

In [97]:
filter_df = filter_and_display(reduced_lre, 'Exonuclease_-_putative')
filter_df

The of filtered data: (4, 17)
The unique values in column 'lre_name': 2
notes
Exonuclease_-_putative    4
Name: count, dtype: int64


,lre_name,lre_family,chrom,gene_id,notes,start,end
349,lre_46.1,46,LinJ.29,LINF_290010600,Exonuclease_-_putative,205977,209251
352,lre_47.1,47,LinJ.29,LINF_290010600,Exonuclease_-_putative,205977,209251


In [98]:
# Check family 46
lre_df.loc[
    (lre_df['gene']==True) & (lre_df['lre_family'] == 46),
    ['lre_name', 'chrom', 'gene_id', 'notes', 'start', 'end']
]

,lre_name,chrom,gene_id,notes,start,end
349,lre_46.1,LinJ.29,LINF_290010600,Exonuclease_-_putative,205977,209251
355,lre_46.2,LinJ.29,LINF_290013800,hypothetical_protein_-_conserved,315865,322387


In [99]:
# Check family 47
lre_df.loc[
    (lre_df['gene']==True) & (lre_df['lre_family'] == 47),
    ['lre_name', 'chrom', 'gene_id', 'notes', 'start', 'end']
]

,lre_name,chrom,gene_id,notes,start,end
352,lre_47.1,LinJ.29,LINF_290010600,Exonuclease_-_putative,205977,209251
358,lre_47.2,LinJ.29,LINF_290013800,hypothetical_protein_-_conserved,315865,322387


In [100]:
# Let's keep them
good_lre, reduced_lre = accept_group(reduced_lre, filter_df, good_lre, reduced_lre)

Data to filter:
Shape of the data frame is: (6, 17)
Number of unique values in column 'lre_name': 2 
Good lre elements:
Shape of the data frame is: (325, 17)
Number of unique values in column 'lre_name': 98 
Keep filtering df:
Shape of the data frame is: (448, 17)
Number of unique values in column 'lre_name': 173 


In [101]:
group_and_count(reduced_lre)

{'gene': 173, 'transcript': 173, '3utr': 64, '5utr': 7, 'CDS': 2}


,notes,count
0,NaN,173
1,Rab-GTPase-TBC_domain_containing_protein,2
2,H(+)-exporting_diphosphatase,2
3,peptidase_m20/m25/m40_family-like_protein,2
4,Vps23_core_domain_containing_protein|Li1040,2
...,...,...
61,tRNA-Leu,1
62,engulfment_and_cell_motility_domain_2_-_putative,1
63,Protein_kinase_domain-containing_protein,1
64,protein-l-isoaspartate_o-methyltransferase_-_p...,1


### Rab-GTPase-TBC_domain_containing_protein

In [102]:
filter_df = filter_and_display(reduced_lre, 'Rab-GTPase-TBC_domain_containing_protein')
filter_df

The of filtered data: (4, 17)
The unique values in column 'lre_name': 2
notes
Rab-GTPase-TBC_domain_containing_protein    4
Name: count, dtype: int64


,lre_name,lre_family,chrom,gene_id,notes,start,end
400,lre_55.1,55,LinJ.30,LINF_300008300,Rab-GTPase-TBC_domain_containing_protein,102141,105323
504,lre_55.2,55,LinJ.33,LINF_330032500,Rab-GTPase-TBC_domain_containing_protein,1005687,1009527


In [103]:
# Check family 55
lre_df.loc[
    (lre_df['gene']==True) & (lre_df['lre_family'] == 55),
    ['lre_name', 'chrom', 'gene_id', 'notes', 'start', 'end']
]

,lre_name,chrom,gene_id,notes,start,end
400,lre_55.1,LinJ.30,LINF_300008300,Rab-GTPase-TBC_domain_containing_protein,102141,105323
403,lre_55.1,LinJ.30,LINF_300008400,KU80_protein,105722,109074
501,lre_55.2,LinJ.33,LINF_330032400,Acetate:succinate_CoA_transferase,1002245,1005303
504,lre_55.2,LinJ.33,LINF_330032500,Rab-GTPase-TBC_domain_containing_protein,1005687,1009527


In [104]:
# Let's keep them
good_lre, reduced_lre = accept_group(reduced_lre, filter_df, good_lre, reduced_lre)

Data to filter:
Shape of the data frame is: (12, 17)
Number of unique values in column 'lre_name': 2 
Good lre elements:
Shape of the data frame is: (337, 17)
Number of unique values in column 'lre_name': 100 
Keep filtering df:
Shape of the data frame is: (436, 17)
Number of unique values in column 'lre_name': 171 


In [105]:
group_and_count(reduced_lre)

{'gene': 171, 'transcript': 171, '3utr': 62, '5utr': 5, 'CDS': 2}


,notes,count
0,NaN,171
1,H(+)-exporting_diphosphatase,2
2,peptidase_m20/m25/m40_family-like_protein,2
3,Vps23_core_domain_containing_protein|Li1040,2
4,vacuolar-type_Ca2+-ATPase_-_putative,2
...,...,...
58,tRNA-Leu,1
59,engulfment_and_cell_motility_domain_2_-_putative,1
60,Protein_kinase_domain-containing_protein,1
61,protein-l-isoaspartate_o-methyltransferase_-_p...,1


### H(+)-exporting_diphosphatase

In [106]:
filter_df = filter_and_display(reduced_lre, 'H\\(\\+\\)\\-exporting_diphosphatase')
filter_df

The of filtered data: (4, 17)
The unique values in column 'lre_name': 2
notes
H(+)-exporting_diphosphatase    4
Name: count, dtype: int64


,lre_name,lre_family,chrom,gene_id,notes,start,end
441,lre_59.2,59,LinJ.31,LINF_310018700,H(+)-exporting_diphosphatase,500464,506449
444,lre_58.6,58,LinJ.31,LINF_310018700,H(+)-exporting_diphosphatase,500464,506449


In [107]:
# Check family 59
lre_df.loc[
    (lre_df['gene']==True) & (lre_df['lre_family'] == 59),
    ['lre_name', 'chrom', 'gene_id', 'notes', 'start', 'end']
]

,lre_name,chrom,gene_id,notes,start,end
441,lre_59.2,LinJ.31,LINF_310018700,H(+)-exporting_diphosphatase,500464,506449


The other members is intergenic

In [108]:
# Check family 58
lre_df.loc[
    (lre_df['gene']==True) & (lre_df['lre_family'] == 58),
    ['lre_name', 'chrom', 'gene_id', 'notes', 'start', 'end']
]

,lre_name,chrom,gene_id,notes,start,end
425,lre_58.1,LinJ.31,LINF_310014250,NaN,317136,317964
427,lre_58.2,LinJ.31,LINF_310014500,amino_acid_permease_3|arginine_transporter|AAP...,322998,328575
430,lre_58.3,LinJ.31,LINF_310016350,NaN,405961,406903
435,lre_58.5,LinJ.31,LINF_310017000,hypothetical_protein_-_conserved,440694,447113
444,lre_58.6,LinJ.31,LINF_310018700,H(+)-exporting_diphosphatase,500464,506449


58.4 es intergénico

In [109]:
# Let's keep them
good_lre, reduced_lre = accept_group(reduced_lre, filter_df, good_lre, reduced_lre)

Data to filter:
Shape of the data frame is: (6, 17)
Number of unique values in column 'lre_name': 2 
Good lre elements:
Shape of the data frame is: (343, 17)
Number of unique values in column 'lre_name': 102 
Keep filtering df:
Shape of the data frame is: (430, 17)
Number of unique values in column 'lre_name': 169 


In [110]:
group_and_count(reduced_lre)

{'gene': 169, 'transcript': 169, '3utr': 60, '5utr': 5, 'CDS': 2}


,notes,count
0,NaN,169
1,peptidase_m20/m25/m40_family-like_protein,2
2,Vps23_core_domain_containing_protein|Li1040,2
3,vacuolar-type_Ca2+-ATPase_-_putative,2
4,Cardiolipin_synthetase,2
...,...,...
57,tRNA-Leu,1
58,engulfment_and_cell_motility_domain_2_-_putative,1
59,Protein_kinase_domain-containing_protein,1
60,protein-l-isoaspartate_o-methyltransferase_-_p...,1


### peptidase_m20/m25/m40_family-like_protein

In [111]:
filter_df = filter_and_display(reduced_lre, 'peptidase_m20/m25/m40_family-like_protein')
filter_df

The of filtered data: (4, 17)
The unique values in column 'lre_name': 2
notes
peptidase_m20/m25/m40_family-like_protein    4
Name: count, dtype: int64


,lre_name,lre_family,chrom,gene_id,notes,start,end
453,lre_62.1,62,LinJ.31,LINF_310026700,peptidase_m20/m25/m40_family-like_protein,967493,971919
464,lre_62.2,62,LinJ.31,LINF_310028200,peptidase_m20/m25/m40_family-like_protein,1029887,1033101


In [112]:
# Check family 62
lre_df.loc[
    (lre_df['gene']==True) & (lre_df['lre_family'] == 62),
    ['lre_name', 'chrom', 'gene_id', 'notes', 'start', 'end']
]

,lre_name,chrom,gene_id,notes,start,end
453,lre_62.1,LinJ.31,LINF_310026700,peptidase_m20/m25/m40_family-like_protein,967493,971919
464,lre_62.2,LinJ.31,LINF_310028200,peptidase_m20/m25/m40_family-like_protein,1029887,1033101


In [113]:
# Let's discard them
reduced_lre = reduced_lre[reduced_lre['lre_family'] != 62]

In [114]:
group_and_count(reduced_lre)

{'gene': 167, 'transcript': 167, '3utr': 58, '5utr': 5, 'CDS': 2}


,notes,count
0,NaN,167
1,Vps23_core_domain_containing_protein|Li1040,2
2,vacuolar-type_Ca2+-ATPase_-_putative,2
3,Cardiolipin_synthetase,2
4,eukaryotic_translation_initiation_factor_2-alp...,2
...,...,...
56,tRNA-Leu,1
57,engulfment_and_cell_motility_domain_2_-_putative,1
58,Protein_kinase_domain-containing_protein,1
59,protein-l-isoaspartate_o-methyltransferase_-_p...,1


### Vps23_core_domain_containing_protein|Li1040

In [115]:
filter_df = filter_and_display(reduced_lre, 'Vps23_core_domain_containing_protein|Li1040')
filter_df

The of filtered data: (4, 17)
The unique values in column 'lre_name': 2
notes
Vps23_core_domain_containing_protein|Li1040    4
Name: count, dtype: int64


,lre_name,lre_family,chrom,gene_id,notes,start,end
481,lre_64.3,64,LinJ.32,LINF_320015500,Vps23_core_domain_containing_protein|Li1040,391856,395482
483,lre_65.3,65,LinJ.32,LINF_320015500,Vps23_core_domain_containing_protein|Li1040,391856,395482


In [116]:
# Check family 64
lre_df.loc[
    (lre_df['gene']==True) & (lre_df['lre_family'] == 64),
    ['lre_name', 'chrom', 'gene_id', 'notes', 'start', 'end']
]

,lre_name,chrom,gene_id,notes,start,end
471,lre_64.1,LinJ.32,LINF_320014830,NaN,345025,351432
475,lre_64.2,LinJ.32,LINF_320015400,hypothetical_protein_-_conserved,374515,391521
481,lre_64.3,LinJ.32,LINF_320015500,Vps23_core_domain_containing_protein|Li1040,391856,395482


In [117]:
# Check family 65
lre_df.loc[
    (lre_df['gene']==True) & (lre_df['lre_family'] == 65),
    ['lre_name', 'chrom', 'gene_id', 'notes', 'start', 'end']
]

,lre_name,chrom,gene_id,notes,start,end
473,lre_65.1,LinJ.32,LINF_320014830,NaN,345025,351432
477,lre_65.2,LinJ.32,LINF_320015400,hypothetical_protein_-_conserved,374515,391521
483,lre_65.3,LinJ.32,LINF_320015500,Vps23_core_domain_containing_protein|Li1040,391856,395482


In [118]:
# Let's keep them
good_lre, reduced_lre = accept_group(reduced_lre, filter_df, good_lre, reduced_lre)

Data to filter:
Shape of the data frame is: (6, 17)
Number of unique values in column 'lre_name': 2 
Good lre elements:
Shape of the data frame is: (349, 17)
Number of unique values in column 'lre_name': 104 
Keep filtering df:
Shape of the data frame is: (418, 17)
Number of unique values in column 'lre_name': 165 


In [119]:
group_and_count(reduced_lre)

{'gene': 165, 'transcript': 165, '3utr': 56, '5utr': 5, 'CDS': 2}


,notes,count
0,NaN,165
1,vacuolar-type_Ca2+-ATPase_-_putative,2
2,Cardiolipin_synthetase,2
3,eukaryotic_translation_initiation_factor_2-alp...,2
4,Dual_specificity_protein_phosphatase,2
5,mitochondrial_carrier_protein_-_putative,1
6,adaptin_complex_1_subunit|beta_adaptin,1
7,tuzin,1
8,Flagellum_attachment_zone_protein_3|FAZ3,1
9,leucine-rich_repeat_protein,1


### vacuolar-type_Ca2+-ATPase_-_putative

In [120]:
filter_df = filter_and_display(reduced_lre, 'vacuolar\\-type_Ca2\\+\\-ATPase_\\-_putative')
filter_df

The of filtered data: (4, 17)
The unique values in column 'lre_name': 2
notes
vacuolar-type_Ca2+-ATPase_-_putative    4
Name: count, dtype: int64


,lre_name,lre_family,chrom,gene_id,notes,start,end
36,lre_7.3,7,LinJ.07,LINF_070012100,vacuolar-type_Ca2+-ATPase_-_putative,297019,304430
38,lre_8.2,8,LinJ.07,LINF_070012100,vacuolar-type_Ca2+-ATPase_-_putative,297019,304430


In [121]:
# Check family 7
lre_df.loc[
    (lre_df['gene']==True) & (lre_df['lre_family'] == 7),
    ['lre_name', 'chrom', 'gene_id', 'notes', 'start', 'end']
]

,lre_name,chrom,gene_id,notes,start,end
28,lre_7.1,LinJ.07,LINF_070010140,NaN,211168,211744
32,lre_7.1,LinJ.07,LINF_070010160,NaN,211951,214227
34,lre_7.2,LinJ.07,LINF_070010950,NaN,251481,252207
36,lre_7.3,LinJ.07,LINF_070012100,vacuolar-type_Ca2+-ATPase_-_putative,297019,304430


In [122]:
# Check family 8
lre_df.loc[
    (lre_df['gene']==True) & (lre_df['lre_family'] == 8),
    ['lre_name', 'chrom', 'gene_id', 'notes', 'start', 'end']
]

,lre_name,chrom,gene_id,notes,start,end
30,lre_8.1,LinJ.07,LINF_070010160,NaN,211951,214227
38,lre_8.2,LinJ.07,LINF_070012100,vacuolar-type_Ca2+-ATPase_-_putative,297019,304430


In [123]:
# Let's keep them
good_lre, reduced_lre = accept_group(reduced_lre, filter_df, good_lre, reduced_lre)

Data to filter:
Shape of the data frame is: (6, 17)
Number of unique values in column 'lre_name': 2 
Good lre elements:
Shape of the data frame is: (355, 17)
Number of unique values in column 'lre_name': 106 
Keep filtering df:
Shape of the data frame is: (412, 17)
Number of unique values in column 'lre_name': 163 


In [124]:
group_and_count(reduced_lre)

{'gene': 163, 'transcript': 163, '3utr': 54, '5utr': 5, 'CDS': 2}


,notes,count
0,NaN,163
1,Cardiolipin_synthetase,2
2,eukaryotic_translation_initiation_factor_2-alp...,2
3,Dual_specificity_protein_phosphatase,2
4,mitochondrial_carrier_protein_-_putative,1
5,adaptin_complex_1_subunit|beta_adaptin,1
6,tuzin,1
7,Flagellum_attachment_zone_protein_3|FAZ3,1
8,leucine-rich_repeat_protein,1
9,histidine_phosphatase,1


### Cardiolipin_synthetase

In [125]:
filter_df = filter_and_display(reduced_lre, 'Cardiolipin_synthetase')
filter_df

The of filtered data: (4, 17)
The unique values in column 'lre_name': 2
notes
Cardiolipin_synthetase    4
Name: count, dtype: int64


,lre_name,lre_family,chrom,gene_id,notes,start,end
620,lre_74.1,74,LinJ.34,LINF_340026200,Cardiolipin_synthetase,889011,895326
623,lre_75.1,75,LinJ.34,LINF_340026200,Cardiolipin_synthetase,889011,895326


In [126]:
# Check family 74
lre_df.loc[
    (lre_df['gene']==True) & (lre_df['lre_family'] == 74),
    ['lre_name', 'chrom', 'gene_id', 'notes', 'start', 'end']
]

,lre_name,chrom,gene_id,notes,start,end
620,lre_74.1,LinJ.34,LINF_340026200,Cardiolipin_synthetase,889011,895326
626,lre_74.2,LinJ.34,LINF_340026700,eukaryotic_translation_initiation_factor_2-alp...,902943,911956
632,lre_74.3,LinJ.34,LINF_340027100,Dual_specificity_protein_phosphatase,923279,937617


In [127]:
# Check family 75
lre_df.loc[
    (lre_df['gene']==True) & (lre_df['lre_family'] == 75),
    ['lre_name', 'chrom', 'gene_id', 'notes', 'start', 'end']
]

,lre_name,chrom,gene_id,notes,start,end
623,lre_75.1,LinJ.34,LINF_340026200,Cardiolipin_synthetase,889011,895326
629,lre_75.2,LinJ.34,LINF_340026700,eukaryotic_translation_initiation_factor_2-alp...,902943,911956
635,lre_75.3,LinJ.34,LINF_340027100,Dual_specificity_protein_phosphatase,923279,937617


In [128]:
# Let's keep them:
good_lre, reduced_lre = accept_group(reduced_lre, filter_df, good_lre, reduced_lre)

Data to filter:
Shape of the data frame is: (6, 17)
Number of unique values in column 'lre_name': 2 
Good lre elements:
Shape of the data frame is: (361, 17)
Number of unique values in column 'lre_name': 108 
Keep filtering df:
Shape of the data frame is: (406, 17)
Number of unique values in column 'lre_name': 161 


In [129]:
group_and_count(reduced_lre)

{'gene': 161, 'transcript': 161, '3utr': 52, '5utr': 5, 'CDS': 2}


,notes,count
0,NaN,161
1,eukaryotic_translation_initiation_factor_2-alp...,2
2,Dual_specificity_protein_phosphatase,2
3,mitochondrial_carrier_protein_-_putative,1
4,adaptin_complex_1_subunit|beta_adaptin,1
5,tuzin,1
6,Flagellum_attachment_zone_protein_3|FAZ3,1
7,leucine-rich_repeat_protein,1
8,histidine_phosphatase,1
9,Surface_antigen_protein_2,1


### eukaryotic_translation_initiation_factor_2-alpha_kinase|eIF2alpha_kinase|PERK|EIF2AK2

In [130]:
filter_df = filter_and_display(reduced_lre, 'eukaryotic_translation_initiation_factor_2\\-alpha_kinase\\|eIF2alpha_kinase\\|PERK\\|EIF2AK2')
filter_df

The of filtered data: (4, 17)
The unique values in column 'lre_name': 2
notes
eukaryotic_translation_initiation_factor_2-alpha_kinase|eIF2alpha_kinase|PERK|EIF2AK2    4
Name: count, dtype: int64


,lre_name,lre_family,chrom,gene_id,notes,start,end
626,lre_74.2,74,LinJ.34,LINF_340026700,eukaryotic_translation_initiation_factor_2-alp...,902943,911956
629,lre_75.2,75,LinJ.34,LINF_340026700,eukaryotic_translation_initiation_factor_2-alp...,902943,911956


Same families as in [cardiolipin_synthetase](#cardiolipin_synthetase)

In [131]:
# Let's keep them:
good_lre, reduced_lre = accept_group(reduced_lre, filter_df, good_lre, reduced_lre)

Data to filter:
Shape of the data frame is: (6, 17)
Number of unique values in column 'lre_name': 2 
Good lre elements:
Shape of the data frame is: (367, 17)
Number of unique values in column 'lre_name': 110 
Keep filtering df:
Shape of the data frame is: (400, 17)
Number of unique values in column 'lre_name': 159 


In [132]:
group_and_count(reduced_lre)

{'gene': 159, 'transcript': 159, '3utr': 50, '5utr': 5, 'CDS': 2}


,notes,count
0,NaN,159
1,Dual_specificity_protein_phosphatase,2
2,mitochondrial_carrier_protein_-_putative,1
3,adaptin_complex_1_subunit|beta_adaptin,1
4,tuzin,1
5,Flagellum_attachment_zone_protein_3|FAZ3,1
6,leucine-rich_repeat_protein,1
7,histidine_phosphatase,1
8,Surface_antigen_protein_2,1
9,Surface_antigenic_protein|PSA-2,1


### Dual_specificity_protein_phosphatase

In [133]:
filter_df = filter_and_display(reduced_lre, 'Dual_specificity_protein_phosphatase')
filter_df

The of filtered data: (4, 17)
The unique values in column 'lre_name': 2
notes
Dual_specificity_protein_phosphatase    4
Name: count, dtype: int64


,lre_name,lre_family,chrom,gene_id,notes,start,end
632,lre_74.3,74,LinJ.34,LINF_340027100,Dual_specificity_protein_phosphatase,923279,937617
635,lre_75.3,75,LinJ.34,LINF_340027100,Dual_specificity_protein_phosphatase,923279,937617


Same families as in [cardiolipin_synthetase](#Cardiolipin_synthetase) and [eukaryotic_translation_initiation_factor_2-alpha_kinaseeif2alpha_kinaseperkeif2ak2](#eukaryotic_translation_initiation_factor_2-alpha_kinaseeif2alpha_kinaseperkeif2ak2)

In [134]:
# Let's keep them
good_lre, reduced_lre = accept_group(reduced_lre, filter_df, good_lre, reduced_lre)

Data to filter:
Shape of the data frame is: (6, 17)
Number of unique values in column 'lre_name': 2 
Good lre elements:
Shape of the data frame is: (373, 17)
Number of unique values in column 'lre_name': 112 
Keep filtering df:
Shape of the data frame is: (394, 17)
Number of unique values in column 'lre_name': 157 


In [135]:
group_and_count(reduced_lre)

{'gene': 157, 'transcript': 157, '3utr': 48, '5utr': 5, 'CDS': 2}


,notes,count
0,NaN,157
1,mitochondrial_carrier_protein_-_putative,1
2,adaptin_complex_1_subunit|beta_adaptin,1
3,tuzin,1
4,Flagellum_attachment_zone_protein_3|FAZ3,1
5,leucine-rich_repeat_protein,1
6,histidine_phosphatase,1
7,Surface_antigen_protein_2,1
8,Surface_antigenic_protein|PSA-2,1
9,phosphoglycan_beta_1-3_galactosyltransferase,1


In [136]:
total = pd.concat([good_lre, reduced_lre])
print(total['lre_name'].nunique())

269
